In [1]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-12.parquet')
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet')

In [3]:
df1.shape, df2.shape

((1461898, 19), (1369769, 19))

In [3]:
df1 = df1.sample(frac=0.1, random_state= 42)
df2 = df2.sample(frac= 0.1, random_state= 42)

In [4]:
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/07/13 18:52:07 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/13 18:52:07 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/experiment_tracking/mlruns/1', creation_time=1752421917144, experiment_id='1', last_update_time=1752421917144, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [ ]:
df1.head()

In [6]:
df1['duration'] = df1.tpep_dropoff_datetime - df1.tpep_pickup_datetime

In [ ]:
df1['duration'].iloc[0]

In [ ]:
df1['duration']

In [ ]:
df1['duration'].sort_values()

In [ ]:
ts = df1['duration'].iloc[0]

In [ ]:
ts

In [ ]:
ts.total_seconds()

In [7]:
df1['duration in min'] = df1['duration'].apply(lambda x: x.total_seconds()/60)

In [ ]:
df1.head()

In [ ]:
sns.distplot(df1['duration in min'])

In [ ]:
df1['duration in min'].describe(percentiles = [0.95,0.98,0.99])

In [8]:
df = df1[(df1['duration in min']>=1)& (df1['duration in min']<=60)].drop(columns = ['duration'], axis = 1)

In [ ]:
df.head()

In [ ]:
sns.distplot(df['duration in min'])

In [9]:
df["PU_DO"] = df['PULocationID'].astype(str) +'_'+df['DOLocationID'].astype(str)

In [10]:
cat = ['PU_DO']
num = ['trip_distance']

In [ ]:
df[cat].dtypes

In [11]:
df[cat] = df[cat].astype(str)

In [12]:
dv = DictVectorizer()

In [ ]:
df[cat+num]

In [ ]:
df[cat].dtypes

In [13]:
df_train = df[cat+num].to_dict(orient = 'records')

In [ ]:
df_train[:10]

In [14]:
X_train = dv.fit_transform(df_train)

In [ ]:
X_train

In [15]:
y_train = df['duration in min'].values

In [ ]:
y_train

In [16]:

lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)



In [17]:
mean_squared_error(y_train, y_pred, squared= False)

4.306819256213457

In [18]:
def preprocess(filename):
    df = pd.read_parquet(filename)
    df = df.sample(frac=0.1, random_state =42)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration in min'] = df['duration'].apply(lambda x: x.total_seconds()/60)
    df = df[(df['duration in min']>=1)& (df['duration in min']<=60)].drop(columns = ['duration'], axis = 1)
    df["PU_DO"] = df['PULocationID'].astype(str) +'_' +df['DOLocationID'].astype(str)
    df['PU_DO'] = df['PU_DO'].astype(str)
    
    return df

In [19]:
df_train = preprocess('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-12.parquet')

In [20]:
df_val = preprocess('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet')

In [26]:
df_train.shape, df_val.shape

((143201, 21), (134381, 21))

In [21]:
train_dict = df_train[cat+num].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dict)
y_train = df_train['duration in min'].values


In [22]:
val_dict = df_val[cat+num].to_dict(orient = 'records')
X_val = dv.transform(val_dict)
y_val = df_val['duration in min'].values


In [23]:

lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

In [24]:
mean_squared_error(y_val, y_pred, squared= False)

5.043634330015768

In [ ]:
import pickle

with open('models/lin_reg.bin', 'wb') as model_out:
    pickle.dump((dv, lr), model_out)

In [26]:
from sklearn.linear_model import Lasso


In [31]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'chaimaa')

    mlflow.log_param('train-data-path', './data/yellow_tripdata_2020-12.parquet')
    mlflow.log_param('valid-data-path','./data/yellow_tripdata_2021-01.parquet')

    alpha = 0.4

    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)
    rmse = mean_squared_error(y_train, y_pred, squared= False)
    mlflow.log_metric('rmse', rmse)
    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle')

In [ ]:

lr2 = Lasso(0.1)
lr2.fit(X_train, y_train)

y_pred2 = lr2.predict(X_val)

mean_squared_error(y_val, y_pred2, squared= False)

In [32]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope




In [33]:
train = xgb.DMatrix(X_train, label= y_train)
valid = xgb.DMatrix(X_val, label= y_val)

In [34]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model','xgboost')
        mlflow.set_tag("developer", 'chaimaa')
        mlflow.log_params(params)

        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds = 50
        )



        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared= False)
        mlflow.log_metric('rmse', rmse)
    return {"loss": rmse, "status": STATUS_OK}

In [35]:
search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 50
}

best_result = fmin(
    fn = objective,
    space = search_space,
    algo= tpe.suggest,
    max_evals= 50,
    trials= Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:15:40] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.56990                           
[1]	validation-rmse:4.80416                           
[2]	validation-rmse:4.68548                           
[3]	validation-rmse:4.66632                           
[4]	validation-rmse:4.65316                           
[5]	validation-rmse:4.65037                           
[6]	validation-rmse:4.64714                           
[7]	validation-rmse:4.63936                           
[8]	validation-rmse:4.63890                           
[9]	validation-rmse:4.63421                           
[10]	validation-rmse:4.63501                          
[11]	validation-rmse:4.63082                          
[12]	validation-rmse:4.63280                          
[13]	validation-rmse:4.62925                          
[14]	validation-rmse:4.63092                          
[15]	validation-rmse:4.62870                          
[16]	validation-rmse:4.62993                          
[17]	validation-rmse:4.62722                          
[18]	valid

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:17:09] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.05181                                                     
[1]	validation-rmse:5.92258                                                     
[2]	validation-rmse:5.24915                                                     
[3]	validation-rmse:4.89473                                                     
[4]	validation-rmse:4.71655                                                     
[5]	validation-rmse:4.62489                                                     
[6]	validation-rmse:4.57899                                                     
[7]	validation-rmse:4.54873                                                     
[8]	validation-rmse:4.54255                                                     
[9]	validation-rmse:4.53847                                                     
[10]	validation-rmse:4.53703                                                    
[11]	validation-rmse:4.53505                                                    
[12]	validation-rmse:4.52891

...

[0]	validation-rmse:6.18880                                                       
[1]	validation-rmse:5.10784                                                       
[2]	validation-rmse:4.75728                                                       
[3]	validation-rmse:4.65722                                                       
[4]	validation-rmse:4.63682                                                       
[5]	validation-rmse:4.63378                                                       
[6]	validation-rmse:4.63156                                                       
[7]	validation-rmse:4.62898                                                       
[8]	validation-rmse:4.62659                                                       
[9]	validation-rmse:4.61791                                                       
[10]	validation-rmse:4.61876                                                      
[11]	validation-rmse:4.61563                                                      
[12]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:25:51] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.55549                                                      
[1]	validation-rmse:8.20997                                                      
[2]	validation-rmse:7.88986                                                      
[3]	validation-rmse:7.59401                                                      
[4]	validation-rmse:7.32100                                                      
[5]	validation-rmse:7.06951                                                      
[6]	validation-rmse:6.83819                                                      
[7]	validation-rmse:6.62604                                                      
[8]	validation-rmse:6.43179                                                      
[9]	validation-rmse:6.25424                                                      
[10]	validation-rmse:6.09236                                                     
[11]	validation-rmse:5.94503                                                     
[12]	validation-

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:33:03] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



: 